In [1]:
using Pkg

#Pkg.add("Optim")
#Pkg.add("PyPlot")
#Pkg.add("LinearAlgebra")

using Optim
using PyPlot
using LinearAlgebra

In [2]:
function market_shares(p,alpha)
    return exp.(-alpha.*p) / (1 + sum( exp.(-alpha.*p) ) )
end


function mco_profits(p,phi,mu,alpha)
    return market_shares(p,alpha) .* ( p - mu .* phi )
end


function price_profits(phi,mu,alpha)
    diff =  5.
    p = [1.,1.]
    profits = [0.,0.]
    
    maxiter = 10
    while (maxiter >=0) & (diff > 10e-7)
        #set up objective
        obj1(p0) = -1*mco_profits( [p0[1], p[2]], phi,mu,alpha)[1]
        res1 = optimize(obj1, [p[1]] , BFGS())
        
        obj2(p0) = -1*mco_profits( [p[1], p0[1]] ,phi,mu,alpha)[2]
        res2 = optimize(obj2,[p[2]], BFGS())
        
        #update loop variables
        diff = max( abs( res1.minimizer[1] - p[1] ), abs( res2.minimizer[1] - p[2] ) )
        maxiter = maxiter-1
        
        #update result
        p = [res1.minimizer[1],res2.minimizer[1]]
        profits =  [-1*res1.minimum, -1*res2.minimum]        
    end
    return p, profits
end


#arbitrary outside option... passive beliefs
function nash_in_nash(phi, mu, alpha; mc=0., pi0=0.)
    p, profits = price_profits(phi,mu,alpha)
    s = market_shares(p,alpha)
    return  profits .* (sum(mu .* s .* (phi .- mc) ) - pi0)
end

print( nash_in_nash( [3,3], [.5,.5] , 1 ) )

[0.0141307, 0.0141307]

In [3]:
function bargain_helper(phi, mu, alpha)
    """solve each firm 1s optimization holding theta 2 fixed"""
    
    obj1(phi0) = -1*nash_in_nash( [phi0[1], phi[2]] , mu, alpha)[1]
    res1 =  optimize(obj1, [phi[1]], BFGS())

    obj2(phi0) = -1*nash_in_nash( [phi[1], phi0[1]] , mu, alpha)[2]
    res2 =  optimize(obj2, [phi[2]], BFGS())
    
    return [ res1.minimizer[1], res2.minimizer[1] ]
end

println( bargain_helper([2.,2.],[.5,.5], 1) )

[0.748419, 0.748419]


In [4]:
function simult_bargain(mu, alpha)
    """solve the bargaining problems seperately and iterate for a solution"""       
    
    #loop variables
    diff =  5.
    phi = [1., 1.]

    maxiter = 10
    while (maxiter >=0) & (diff > 10e-7)
        #seems as though there is a contraction mapping here, need to think more about why
        phi_new = bargain_helper(phi, mu, alpha)
        
        #update loop variables
        diff = maximum( abs.(phi - phi_new) )
        maxiter = maxiter-1
        
        #update phi
        phi = phi_new
    end
    return phi
end
    
print(simult_bargain([.5,.5], 1.5) )

[0.572179, 0.572179]

In [5]:
function bargain_helper_seq(phi1, mu, alpha)
    """ solve firm 1s optimization holding theta 2 fixed """
    obj1(phi2) = -1*nash_in_nash( [ phi1, phi2[1] ] , mu, alpha)[2]
    res =  optimize(obj1, [2.], BFGS())
    return res.minimizer[1]
end


function seq_obj(phi1, mu, alpha)
    """ theta1 is a guess for the first stage """
    phi2 = bargain_helper_seq(phi1, mu, alpha) 
    pi10 = mu[1]*phi1*exp(-alpha * phi1)/(1+exp(-alpha *phi1))
    return nash_in_nash([phi1, phi2], mu, alpha, pi0=pi10)[1]
end

bargain_helper_seq(1, [.5,.5], 1)

seq_obj(1, [.5,.5], 1)

-0.0008447225201653179

In [6]:
function seq_bargain(mu, alpha)
    """solve the bargaining problems seperately,
    then solve for the solution"""
    
    obj1(phi10) = -1*seq_obj(phi10[1], mu, alpha)
    res = optimize(obj1, [2.], BFGS())
    phi1  = res.minimizer[1]
    phi2 = bargain_helper([phi1,2.], mu, alpha)[1]
    return [phi1, phi2]
end

print(seq_bargain([.5,.5], .1))

[-7016.16, 10295.6]

In [11]:
function make_table(steps, mu)
    """the argument is 
    steps = min_arg, max_arg, no_step """
    
    min_arg, max_arg, no_step = steps
    
    cols = ["eq_type","listprice","distance_cost","mc1","mc2","profit1","profit2","hosp_profit","share1","share2"]
    
    step_size = (max_arg - min_arg)/no_step
    
    indep_var = min_arg:step_size:max_arg
    
    for eq_type= ["Simultaneous", "Sequential"]
        
        result1 = []
        result2 = []
        
        for alpha=indep_var
            
            phi = [0.,0.]
            
            if eq_type == "Simultaneous"
                phi = simult_bargain(mu, alpha)
            else
                phi = seq_bargain(mu, alpha)
            end


            price, profits = price_profits(phi,mu,alpha)
            share = market_shares(price, alpha)
            hosp = sum(mu .* share .* phi)
            
            append!(result1, [[alpha, phi[1], share[1], price[1], profits[1], hosp ]] ) 
            append!(result2, [[alpha, phi[2], share[2], price[2], profits[2], hosp]] ) 
        end
        
        println("\n",eq_type, "\n" )
        for line =result1
            print( round(line[1],sigdigits=3) )
            for i=2:6
                print( " & ")
                print(round(line[i],sigdigits=3) )
            end
            print(" \\\\\\\\ \n")
        end   
        print('\n')
        
        for line =result2
            print( round(line[1],sigdigits=3) )
            for i=2:6
                print( " & ") 
                print(round(line[i],sigdigits=3) )
            end
            print(" \\\\\\\\ \n")
        end
    end
    

end

    
make_table( (.5,1,5), [.5,.5] )

[0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
Simultaneous

0.5 & 1.72 & 0.144 & 3.19 & 0.337 & 0.247 \\\\ 
0.6 & 1.43 & 0.144 & 2.66 & 0.281 & 0.206 \\\\ 
0.7 & 1.23 & 0.144 & 2.28 & 0.24 & 0.177 \\\\ 
0.8 & 1.07 & 0.144 & 2.0 & 0.21 & 0.155 \\\\ 
0.9 & 0.954 & 0.144 & 1.77 & 0.187 & 0.137 \\\\ 
1.0 & 0.858 & 0.144 & 1.6 & 0.168 & 0.124 \\\\ 

0.5 & 1.72 & 0.144 & 3.19 & 0.337 & 0.247 \\\\ 
0.6 & 1.43 & 0.144 & 2.66 & 0.281 & 0.206 \\\\ 
0.7 & 1.23 & 0.144 & 2.28 & 0.24 & 0.177 \\\\ 
0.8 & 1.07 & 0.144 & 2.0 & 0.21 & 0.155 \\\\ 
0.9 & 0.954 & 0.144 & 1.77 & 0.187 & 0.137 \\\\ 
1.0 & 0.858 & 0.144 & 1.6 & 0.168 & 0.124 \\\\ 


┌ Warning: Linesearch failed, using alpha = 1.081462365426491e-12 and exiting optimization.
│ The linesearch exited with message:
│ Linesearch failed to converge, reached maximum iterations 50.
└ @ Optim /home/erichschulman/.julia/packages/Optim/Agd3B/src/utilities/perform_linesearch.jl:47



Sequential

0.5 & 2.0 & 0.137 & 3.32 & 0.316 & 0.258 \\\\ 
0.6 & 5.07 & 0.059 & 4.31 & 0.104 & 0.256 \\\\ 
0.7 & 4.36 & 0.0586 & 3.7 & 0.089 & 0.217 \\\\ 
0.8 & 3.8 & 0.0589 & 3.23 & 0.0782 & 0.189 \\\\ 
0.9 & 3.37 & 0.0591 & 2.87 & 0.0698 & 0.167 \\\\ 
1.0 & 3.01 & 0.0595 & 2.57 & 0.0633 & 0.15 \\\\ 

0.5 & 1.66 & 0.147 & 3.17 & 0.344 & 0.258 \\\\ 
0.6 & 1.33 & 0.159 & 2.65 & 0.316 & 0.256 \\\\ 
0.7 & 1.11 & 0.161 & 2.26 & 0.274 & 0.217 \\\\ 
0.8 & 0.952 & 0.162 & 1.97 & 0.241 & 0.189 \\\\ 
0.9 & 0.836 & 0.162 & 1.74 & 0.215 & 0.167 \\\\ 
1.0 & 0.748 & 0.162 & 1.57 & 0.194 & 0.15 \\\\ 
